In [ ]:
import numpy as np
from keras.utils.np_utils import to_categorical
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler


In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from keras import models
from keras import layers
from keras import regularizers
from keras.layers import Dropout


In [ ]:
X = np.load('/content/drive/My Drive/Colab Notebooks/ML/New_version_copd/X2_data.npy')
np.random.shuffle(X)
y = X[:,-1]
X=X[:,:-1]

In [ ]:
X.shape

(4432, 72000)

In [ ]:
kfold = KFold(5, True, 1)

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
X = scaler.fit_transform(X)

In [ ]:
import time
import keras


class TimeHistory(keras.callbacks.Callback):

    def on_train_begin(self, logs={}):

        self.times = []

    def on_epoch_begin(self, batch, logs={}):

        self.epoch_time_start = time.time()

    def on_epoch_end(self, batch, logs={}):

        self.times.append(time.time() - self.epoch_time_start)

In [ ]:
X_test_list=[]
y_test_list=[]
test_loss=[]
train_loss=[]
test_acc=[]
train_acc=[]
test_auc=[]
train_auc=[]

for train_index, test_index in kfold.split(X):
  X_train= X[train_index]
  y_train=y[train_index]
  X_test=X[test_index]
  y_test=y[test_index]
  one_hot_train_labels = to_categorical(y_train)
  one_hot_test_labels = to_categorical(y_test)
  X_test_list.append(X_test)
  y_test_list.append(one_hot_test_labels)
  X_train = X_train.reshape(X_train.shape[0],X_train.shape[1], 1)
  X_test = X_test.reshape(X_test.shape[0],X_test.shape[1], 1)
  
  model = models.Sequential()
  model.add(layers.Conv1D(32, kernel_size=3,padding = 'same',activation='relu', input_shape=(X_train.shape[1],1)))
  model.add(layers.Flatten())
  model.add(layers.Dense(30, activation='relu'))
  model.add(layers.Dense(5, activation='softmax'))




  model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy', 'AUC'])
  time_callback = TimeHistory()
  history = model.fit(X_train, one_hot_train_labels, epochs=500, batch_size=64, callbacks=[time_callback],validation_data=(X_test, one_hot_test_labels))
  times = time_callback.times 
  loss = history.history['loss']
  val_loss = history.history['val_loss']
  train_loss.append(loss)
  test_loss.append(val_loss)
  train_acc_1=history.history['accuracy']
  val_acc = history.history['val_accuracy']
  test_acc.append(val_acc)
  train_acc.append(train_acc_1)
  a=history.history['auc']
  b=history.history['val_auc']

  test_auc.append(b)
  train_auc.append(a)
  
  model.save('/content/drive/My Drive/Colab Notebooks/ML/outputs/model_kfold_cnn_'+str(i)+'.h5')
  np.save('/content/drive/My Drive/Colab Notebooks/ML/outputs/train_loss_cnn_k_'+str(i)+'.npy', train_loss)
  np.save('/content/drive/My Drive/Colab Notebooks/ML/outputs/test_loss_cnn_k_'+str(i)+'.npy', test_loss)
  np.save('/content/drive/My Drive/Colab Notebooks/ML/outputs/train_acc_cnn_k_'+str(i)+'.npy', train_acc)
  np.save('/content/drive/My Drive/Colab Notebooks/ML/outputs/test_acc_cnn_k_'+str(i)+'.npy', test_acc)
  np.save('/content/drive/My Drive/Colab Notebooks/ML/outputs/train_auc_cnn_k_'+str(i)+'.npy', train_auc)
  np.save('/content/drive/My Drive/Colab Notebooks/ML/outputs/test_auc_cnn_k_'+str(i)+'.npy', test_auc)

Epoch 1/500
56/56 [==============================] - 155s 2s/step - loss: 82.7587 - accuracy: 0.2613 - auc: 0.5425 - val_loss: 1.6011 - val_accuracy: 0.3630 - val_auc: 0.6311
Epoch 2/500
56/56 [==============================] - 137s 2s/step - loss: 1.5981 - accuracy: 0.3604 - auc: 0.6236 - val_loss: 1.5864 - val_accuracy: 0.3630 - val_auc: 0.6315
Epoch 3/500
56/56 [==============================] - 136s 2s/step - loss: 1.5851 - accuracy: 0.3529 - auc: 0.6224 - val_loss: 1.5735 - val_accuracy: 0.3630 - val_auc: 0.6311
Epoch 4/500
56/56 [==============================] - 138s 2s/step - loss: 1.5723 - accuracy: 0.3556 - auc: 0.6267 - val_loss: 1.5631 - val_accuracy: 0.3630 - val_auc: 0.6311
Epoch 5/500
56/56 [==============================] - 138s 2s/step - loss: 1.5671 - accuracy: 0.3476 - auc: 0.6172 - val_loss: 1.5541 - val_accuracy: 0.3630 - val_auc: 0.6311
Epoch 6/500
56/56 [==============================] - 138s 2s/step - loss: 1.5578 - accuracy: 0.3443 - auc: 0.6233 - val_loss: 1.5

KeyboardInterrupt: ignored